# Accessing quality-controlled historical weather station data

The <span style="color:#FF0000">[Historical Observations Data Platform](https://eaglerockanalytics.com/project/historical-observations-data-platform/)</span> is a cloud-based historical weather observations database that enables access to high-quality, open climate and weather data. The Platform responds to a broad-scale need to understand weather and climate information including the severity, duration, frequency, and rate of change over time of extreme weather events, as well as supporting projections downscaling efforts. The Platform implements stringent, customized Quality Assurance / Quality Control (QA/QC) protocols in line with international conventions, with updates relevant to the energy sector to accurate capture extremes. The Platform has sourced publicly accessible weather observation stations from 27 networks throughout western North America, with a total of **14,927 quality-controlled and standardized stations** spanning 1980-2022. 

For more information on the QA/QC and standardization process, please check out the open-access methods and code at the <span style="color:#FF0000">[Historical Observations Data Platform code repository](https://github.com/Eagle-Rock-Analytics/historical-obs-platform)</span>. A station list of all available quality-controlled and standardized stations is available in <span style="color:#FF0000">[climakitae](https://github.com/cal-adapt/climakitae/blob/data/add-hdp-stnlist/climakitae/data/historical_wx_stations.csv)</span>.

In [ ]:
import intake 

First, open the catalog using `intake`

In [ ]:
cat = intake.open_esm_datastore("https://cadcat.s3.amazonaws.com/histwxstns/era-hdp-collection.json")

Next, view the catalog in table format. You can inspect the first few rows by calling `.head()` on the table.

In [ ]:
# Access catalog as dataframe and inspect the first few rows
cat_df = cat.df
cat_df

View all the weather station networks by using the following code 

In [ ]:
# See all network options 
cat_df["network_id"].unique()

You can also filter the catalog to see all stations within a network

In [ ]:
my_network = "ASOSAWOS"
cat_df[cat_df["network_id"] == my_network]

You can subset the catalog and read in the cloud-optimized data as `xarray.Dataset` objects using the method shown below. To change the data downloaded, simply modify the inputs in the dictionary `query`. These inputs must correspond to valid options in the catalog. 

In [ ]:
# Set your query here 
query = {
    "network_id": "ASOSAWOS",  # Name of the network
    "station_id": ["ASOSAWOS_72288023152","ASOSAWOS_72389093193","ASOSAWOS_72297603166","ASOSAWOS_72493023230"] # List of stations to get data for 
}

# Subset catalog 
cat_subset = cat.search(**query)

# View the data you've selected before downloading
cat_subset.df

Then, you can download all the files. The files will be downloaded as a dictionary, in which each key is a string description of the data, and the item is the data object. 

In [ ]:
# Get dataset dictionary 
dsets = cat_subset.to_dataset_dict(
    xarray_open_kwargs={'consolidated':False},
    storage_options={'anon':True}
)

To see all the string IDs for the Datasets in the dictionary, you can print them with the following code: 

In [ ]:
list(dsets.keys())

You can easily access the files in the dictionary using the following format: 
```
dsets[<string ID of data>]
```
The string ID of the data is constructed using both the network ID and the station ID for each individual weather station. 

In [ ]:
# Retrieve a single file
ds = dsets["ASOSAWOS.ASOSAWOS_72389093193"]
ds

## Make a quick plot of the data 
`xarray` has some nice mapping features that enable you to quickly generate a plot for a single timestep. This lets you get a sense for the data you read in. 

In [ ]:
variable_to_plot = "tas"
ds.squeeze()[variable_to_plot].plot(x="time");

## Subset the historical weather stations for a region

If you're interested in historical weather observation stations in a specific area, you can also subset the full archive of stations to identify those that you are interested in. We will read in the Historical Data Platform station list, which provides the coordinates, dates of coverage, source network, and *total number of observations* for each station. We'll soon be adding additional information like the state and common station identifiers!

For this, we'll use an area shapefile; upload your own to see which stations are in your area! You can pass either a **web link to an open access shapefile**, or **upload your own shapefile** to your Hub instance too. 

In [ ]:
# Read in the Historical Data Platform station list
import geopandas as gpd
from climakitae.util.utils import read_csv_file, clip_gpd_to_shapefile

hdp_stns = read_csv_file("data/historical_wx_stations.csv")

In [ ]:
# Region of Interest shapefile
roi = gpd.read_file("...") # Replace this with your own shapefile!

# If there are multiple polygons within the shapefile and you want to further subset, you will need to do so.
# Example
# roi = roi[roi["NAME"] == "SPECIFIC_AREA"]

In [ ]:
# Clip the stationlist to subset within your area of interest
stns_within_area = clip_gpd_to_shapefile(hdp_stns, roi)

This subset are the stations within the designated area from your submitted shapefile! You can easily export this list now for your own information, and use it to look up specific stations. 

In [ ]:
# Export the subset portion of station list
stns_within_area.to_csv("subset_station_list.csv")

Want a more interactive way to zoom in on an area? use `stns.explore()`! Note, it may take some time to load. 

In [ ]:
stns_within_area.explore()